In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_11_26.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_14_31.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_25_26.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_21_27.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_15_03.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_07_17.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_16_02.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_14_26.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_19_09.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_16_20.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_21_13.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_03_13.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_11_13.tif
/kaggle/input/aksu-dataset1/Aksu/Validation/Mask/GT_colored_25_18.tif
/kaggle/input/aksu-d

In [3]:
!pip install rasterio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import rasterio  # For TIFF images
import os
from tqdm import tqdm
import torchvision.transforms as transforms




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 45.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Temporal Convolutional Block (TCBlock)
class TCBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TCBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x3 = F.relu(self.conv3(x2))
        return self.bn(x3)

# TCNet Model
class TCNet(nn.Module):
    def __init__(self, num_classes):
        super(TCNet, self).__init__()
        self.encoder1 = TCBlock(in_channels=4, out_channels=64)   # 4-channel input
        self.encoder2 = TCBlock(64, 128)
        self.encoder3 = TCBlock(128, 256)

        self.fc1 = None  # Placeholder for dynamic computation
        self.fc2 = None  

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, kernel_size=2))
        e3 = self.encoder3(F.max_pool2d(e2, kernel_size=2))

        # Debug: Print feature map size before flattening
        # print("Feature map shape before flattening:", e3.shape)  # (batch_size, 256, H, W)

        # Dynamically determine FC input size
        if self.fc1 is None:
            flatten_size = e3.view(e3.size(0), -1).size(1)  # Compute exact feature size
            self.fc1 = nn.Linear(flatten_size, 512).to(x.device)
            self.fc2 = nn.Linear(512, num_classes).to(x.device)

        e3_flat = torch.flatten(e3, start_dim=1)  # Flatten for FC layer
        fc1_out = F.relu(self.fc1(e3_flat))
        output = self.fc2(fc1_out)

        return output

# Example Usage
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TCNet(num_classes=10).to(device)

    # Create a random 4-channel input tensor (batch_size=4, channels=4, height=224, width=224)
    dummy_input = torch.randn(4, 4, 224, 224).to(device)

    # Forward pass
    output = model(dummy_input)
    # print("Output shape:", output.shape)  # Should be (4, num_classes)


In [5]:
import os
import cv2
import torch
import numpy as np
from torch.utils import data
from torchvision import transforms

class MM_Dataset(data.Dataset):
    def __init__(self, dataset_dir, mode="train", transform=None):
        self.dataset_dir = dataset_dir
        self.mode = mode
        self.image_paths = []
        self.labels = []

        # Default transform (Handles 4-channel images)
        if transform is None:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((224, 224)),  # Ensure consistency
                transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5, 0.5])  # Updated for 4 channels
            ])
        else:
            self.transform = transform

        # Validate mode path
        mode_path = os.path.join(self.dataset_dir, mode)
        if not os.path.exists(mode_path):
            raise FileNotFoundError(f"Dataset path not found: {mode_path}")

        # Load image paths and labels
        for class_name in sorted(os.listdir(mode_path)):
            class_dir = os.path.join(mode_path, class_name)
            if os.path.isdir(class_dir):
                for file_name in os.listdir(class_dir):
                    file_path = os.path.join(class_dir, file_name)
                    self.image_paths.append(file_path)
                    self.labels.append(class_name)

        # Map labels to numerical indices
        self.label_to_idx = {label: idx for idx, label in enumerate(sorted(set(self.labels)))}
        self.labels = [self.label_to_idx[label] for label in self.labels]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load the image using OpenCV (supports TIFF)
        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            raise RuntimeError(f"Error reading image: {img_path}")

        # Ensure image has 4 channels
        if len(image.shape) == 2:  # Grayscale case
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGBA)
        elif image.shape[-1] == 3:  # Convert 3-channel RGB to 4-channel RGBA
            alpha_channel = np.ones((image.shape[0], image.shape[1], 1), dtype=image.dtype) * 255
            image = np.concatenate((image, alpha_channel), axis=-1)

        # Normalize based on data type
        if image.dtype == np.uint16:
            image = image.astype(np.float32) / 65535.0  # Normalize 16-bit images
        else:
            image = image.astype(np.float32) / 255.0  # Normalize 8-bit images

        # Apply transformations
        image = self.transform(image)

        return image, label


In [6]:
# Define dataset path
dataset_path = "/kaggle/input/aksu-dataset1/Aksu"

# Create datasets
train_dataset = MM_Dataset(dataset_path, mode="Train")
val_dataset = MM_Dataset(dataset_path, mode="Validation")
test_dataset = MM_Dataset(dataset_path, mode="Test")

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

# Check sample batch
sample_batch = next(iter(train_loader))
print("Sample batch shape:", sample_batch[0].shape, "Labels:", sample_batch[1])


Sample batch shape: torch.Size([4, 4, 224, 224]) Labels: tensor([0, 1, 1, 1])


In [7]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Device configuration
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Number of output classes (binary segmentation = 1)
# num_classes = 1

# # Initialize model
# model = TCNet(num_classes).to(device)

# # Define loss function & optimizer
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training function
# def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0

#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # Ensure shape matches output

#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()

#         # Validation
#         model.eval()
#         val_loss = 0.0
#         with torch.no_grad():
#             for images, labels in val_loader:
#                 images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()

#         print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}")

#     # print("Training complete!")

# # Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Number of output classes (Binary segmentation = 1)
num_classes = 1

# Initialize model
model = TCNet(num_classes).to(device)

# Define loss function & optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # Ensure shape matches output

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.sigmoid(outputs).detach().cpu().numpy() > 0.5  # Convert logits to binary
            all_preds.extend(preds.flatten())
            all_labels.extend(labels.cpu().numpy().flatten())

        train_loss = running_loss / len(train_loader)
        train_acc = accuracy_score(all_labels, all_preds)
        train_precision = precision_score(all_labels, all_preds, zero_division=1)

        # Validation
        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
                val_preds.extend(preds.flatten())
                val_labels.extend(labels.cpu().numpy().flatten())

        val_loss /= len(val_loader)
        val_acc = accuracy_score(val_labels, val_preds)
        val_precision = precision_score(val_labels, val_preds, zero_division=1)

        epoch_time = time.time() - start_time

        print(f"Epoch [{epoch+1}/{num_epochs}] - Time: {epoch_time:.2f}s")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, Precision: {train_precision:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, Precision: {val_precision:.4f}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)



Epoch [1/10] - Time: 205.34s
  Train Loss: 0.1807, Accuracy: 0.9499, Precision: 1.0000
  Val Loss: 0.3774, Accuracy: 0.5833, Precision: 0.5455
Epoch [2/10] - Time: 204.59s
  Train Loss: 0.1011, Accuracy: 0.9749, Precision: 1.0000
  Val Loss: 0.1714, Accuracy: 0.9750, Precision: 1.0000
Epoch [3/10] - Time: 205.35s
  Train Loss: 0.0722, Accuracy: 0.9797, Precision: 1.0000
  Val Loss: 0.1353, Accuracy: 0.9750, Precision: 1.0000
Epoch [4/10] - Time: 204.44s
  Train Loss: 0.0632, Accuracy: 0.9833, Precision: 1.0000
  Val Loss: 0.1893, Accuracy: 0.9833, Precision: 1.0000
Epoch [5/10] - Time: 204.99s
  Train Loss: 0.0479, Accuracy: 0.9857, Precision: 1.0000
  Val Loss: 0.0934, Accuracy: 0.9875, Precision: 1.0000
Epoch [6/10] - Time: 194.30s
  Train Loss: 0.0299, Accuracy: 0.9940, Precision: 1.0000
  Val Loss: 0.0791, Accuracy: 0.9875, Precision: 1.0000
Epoch [7/10] - Time: 200.67s
  Train Loss: 0.0261, Accuracy: 0.9952, Precision: 1.0000
  Val Loss: 0.0867, Accuracy: 0.9583, Precision: 1.0000